<a href="https://colab.research.google.com/github/TolulopeMoyo/codelineio/blob/master/smith_Watermann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import itertools
import numpy as np

def matrix(a, b, match_score=3, gap_cost=2):
    H = np.zeros((len(a) + 1, len(b) + 1), np.int)

    for i, j in itertools.product(range(1, H.shape[0]), range(1, H.shape[1])):
        match = H[i - 1, j - 1] + (match_score if a[i - 1] == b[j - 1] else - match_score)
        delete = H[i - 1, j] - gap_cost
        insert = H[i, j - 1] - gap_cost
        H[i, j] = max(match, delete, insert, 0)
    return H

In [0]:
def traceback(H, b, b_='', old_i=0):
    # flip H to get index of **last** occurrence of H.max() with np.argmax()
    H_flip = np.flip(np.flip(H, 0), 1)
    i_, j_ = np.unravel_index(H_flip.argmax(), H_flip.shape)
    i, j = np.subtract(H.shape, (i_ + 1, j_ + 1))  # (i, j) are **last** indexes of H.max()
    if H[i, j] == 0:
        return b_, j
    b_ = b[j - 1] + '-' + b_ if old_i - i > 1 else b[j - 1] + b_
    return traceback(H[0:i, 0:j], b, b_, i)

In [0]:
def smith_waterman(a, b, match_score=3, gap_cost=2):
    a, b = a.upper(), b.upper()
    H = matrix(a, b, match_score, gap_cost)
    b_, pos = traceback(H, b)
    return pos, pos + len(b_)

In [10]:
# prints correct scoring matrix from Wikipedia example
    print(matrix('TACGGGCCCGCTAC', 'TAGCCCTATCGGTCA'))

    a, b = 'tacgggcccgctac', 'tagccctatcggtca'
    H = matrix(a, b)
    print(traceback(H, b)) # ('gtt-ac', 1)

    a, b = 'tacgggcccgctac', 'tagccctatcggtca'
    start, end = smith_waterman(a, b)
    print(a[start:end])     # GTTGAC

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  3  1  0  0  0  0  3  1  3  1  0  0  3  1  0]
 [ 0  1  6  4  2  0  0  1  6  4  2  0  0  1  0  4]
 [ 0  0  4  3  7  5  3  1  4  3  7  5  3  1  4  2]
 [ 0  0  2  7  5  4  2  0  2  1  5 10  8  6  4  2]
 [ 0  0  0  5  4  2  1  0  0  0  3  8 13 11  9  7]
 [ 0  0  0  3  2  1  0  0  0  0  1  6 11 10  8  6]
 [ 0  0  0  1  6  5  4  2  0  0  3  4  9  8 13 11]
 [ 0  0  0  0  4  9  8  6  4  2  3  2  7  6 11 10]
 [ 0  0  0  0  3  7 12 10  8  6  5  3  5  4  9  8]
 [ 0  0  0  3  1  5 10  9  7  5  3  8  6  4  7  6]
 [ 0  0  0  1  6  4  8  7  6  4  8  6  5  3  7  5]
 [ 0  3  1  0  4  3  6 11  9  9  7  5  3  8  6  4]
 [ 0  1  6  4  2  1  4  9 14 12 10  8  6  6  5  9]
 [ 0  0  4  3  7  5  4  7 12 11 15 13 11  9  9  7]]
('ta-g-cc-ac', 0)
tacgggcccg
